In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import *


### **Data Reading**

In [0]:
df = spark.read.format("parquet")\
        .load("abfss://bronze@etedatabricks01.dfs.core.windows.net/customers")
display(df)

In [0]:
df = df.drop("_rescued_data")
display(df)

In [0]:
df = df.withColumn("domains", split(col("email"), "@")[1])
df.display()

In [0]:
df.groupBy("domains").agg(count("customer_id").alias("total_customers")).sort("total_customers", ascending=False).display()

In [0]:
df = df.withColumn("full_name", concat(col("first_name"), lit(" "),  col("last_name")))
df = df.drop("first_name", "last_name")
df.display()

In [0]:
df.write.mode("overwrite").format("delta").save("abfss://silver@etedatabricks01.dfs.core.windows.net/customers") 

In [0]:
%sql
CREATE TABLE IF NOT EXISTS databricks_catalog.silver.customers_silver 
USING DELTA
LOCATION 'abfss://silver@etedatabricks01.dfs.core.windows.net/customers'